### Submission Information:
- Name: Đinh Văn An
- StudentID: K204110855
- Course: BIG DATA AND ITS APPLICATIONS
- Instructor: TS. Nguyễn Thôn Dã
- Assignment: Twitter API

| Syntax      | Description | Test Text     |
| :---        |    :--:   |          ---: |
| Header      | Title       | Here's this   |
| Paragraph   | Text        | And more      |

## Import libraries
> tweepy

> tdqm

> pandas

In [1]:
# import tweepy
import tweepy as tw
from tqdm import tqdm
import pandas as pd

## Part 1: Twitter API credentials
> API Key: 0OPz30qvOsuUa5wbDicAmA8tr

> API Secret: AZ0aQx0ySWfcG56oDptKWJGAAbdzIUWwZuNyIuZQVBEs6DfWOx

In [2]:
# your Twitter API key and API secret
my_api_key = "0OPz30qvOsuUa5wbDicAmA8tr"
my_api_secret = "AZ0aQx0ySWfcG56oDptKWJGAAbdzIUWwZuNyIuZQVBEs6DfWOx"
# authenticate
auth = tw.OAuthHandler(my_api_key, my_api_secret)
api = tw.API(auth, wait_on_rate_limit=True)

## Part 2: Collect the tweets

### 2.1. Set up search query
> Can change the keyword, itemQuantity, language, and filter

- keyword: covid19, president trump, etc.
- itemQuantity: 100, 1000, 10000, etc.
- language: 'vi','en','zhCN', 'de', 'sv', 'ru', 'ko', 'it', 'fr', 'es', 'th', 'ja', 'pl', 'nl'
- filter: 'retweets', 'images', 'videos', 'links', 'replies'

In [3]:
# keywords to search
keyword = "covid-19"
# number of tweets to search
itemQuantity = 50
# language
language = "en"
# search query
search_query = f"#{keyword} -filter:retweets"

### 2.2. Collect

In [4]:
# get tweets from the API
# search for tweets in last year
tweets = tw.Cursor(api.search_tweets, 
                    q=search_query, 
                    lang=language, 
                    since="2020-01-01", 
                    tweet_mode="extended").items(itemQuantity)

# store the API responses in a list
tweets_copy = []
for tweet in tqdm(tweets, desc='Appending', unit=' tweets', total=itemQuantity):
    tweets_copy.append(tweet)
    
print("Total Tweets fetched:", len(tweets_copy))

Appending: 100%|██████████| 50/50 [00:02<00:00, 19.61 tweets/s]

Total Tweets fetched: 50


## Part 3: Save the tweets into DataFrame

### 3.1. Save the tweets into DataFrame

In [5]:
# intialize the dataframe
tweets_df = pd.DataFrame()
# populate the dataframe
for tweet in tqdm(tweets_copy, desc='Saving', unit=' tweets', total=len(tweets_copy)):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = pd.concat([tweets_df, pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,
                                               'user_description': tweet.user.description,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source})], ignore_index=True)

    tweets_df = tweets_df.reset_index(drop=True)

Saving: 100%|██████████| 50/50 [00:22<00:00,  2.25 tweets/s]


### 3.2. Show DataFrame

In [6]:
# show the dataframe
tweets_df.head(10)

,user_name,user_location,user_description,user_verified,date,text,hashtags,source
0,Andrew Havranek,"Milwaukee, WI","Reporter @SpectrumNews1WI • Formerly @KYTV, @W...",True,2022-10-07 16:28:50+00:00,Dorow said the test result document is staying...,[DarrellBrooks],TweetDeck
1,J0K3R,,(pro science) (shill) (bot) \n🇺🇦\n#Anonymous #...,False,2022-10-07 16:28:43+00:00,@boxfromhell @BeenWronged @disclosetv https://...,None,Twitter for Android
2,shovas,Canada,"A sinner, saved. ""For God so loved the world t...",False,2022-10-07 16:28:43+00:00,Canada is now 97.1% Omicron BA.4/5\n\nVaccines...,"[onhealth, onpoli, covid19, cdnhealth, cdnpoli]",Twitter Web App
3,Yahoo News,New York City,Top stories. Trusted sources. Your window to t...,True,2022-10-07 16:28:40+00:00,A rebound of COVID-19 symptoms in some patient...,None,SocialFlow
4,El Calavero Restaurado #ReloadTheSnyderverse,,"Hãy sống là chính mình, bình thường nhưng khôn...",False,2022-10-07 16:28:38+00:00,DiamondLaunch Bybit_Official COVID 19 has slow...,None,Twitter Web App
5,BLAZE 🌻,,she/her “What do we say to the god of death?” ...,False,2022-10-07 16:28:31+00:00,I took a COVID-19 test and I am negative! Hall...,None,Twitter for iPhone
6,Prince,United States,,False,2022-10-07 16:28:22+00:00,The COVID-19 pandemic brought with it new anxi...,None,Twitter for Android
7,JoJo Abbasi,"Dillon, CO",Former NBC Chicago journalist gone to the dark...,False,2022-10-07 16:28:17+00:00,The changing nature of global healthcare consu...,None,dlvr.it
8,Alison Brunier,Geneva,"Public health communications expert, with a pa...",False,2022-10-07 16:28:12+00:00,A fitting tribute to a very special colleague:...,None,Twitter Web App
9,honurable member,,I am not special…others are,False,2022-10-07 16:28:07+00:00,@mmodiba10 Clearly celeb status has acutely co...,None,Twitter for iPhone


### 3.3. Statistics of the tweets DataFrame

In [7]:
tweets_df.describe(include='all', datetime_is_numeric=True)

,user_name,user_location,user_description,user_verified,date,text,hashtags,source
count,50,50,50,50,50,50,8,50
unique,49,30,42,2,NaN,50,8,12
top,Edwin Hayward 🦄 🗡,,,False,NaN,Dorow said the test result document is staying...,[DarrellBrooks],Twitter Web App
freq,2,18,8,43,NaN,1,1,20
mean,NaN,NaN,NaN,NaN,2022-10-07 16:26:31.720000256+00:00,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,2022-10-07 16:24:04+00:00,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,2022-10-07 16:25:24.500000+00:00,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,2022-10-07 16:26:19.500000+00:00,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,2022-10-07 16:27:47.249999872+00:00,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,2022-10-07 16:28:50+00:00,NaN,NaN,NaN
